# BatFlightAnalysis

## Paquetes

In [12]:
import pandas as pd
import numpy as np

from BatFlightAnalysis import read_data, split_data

## Código

### Lectura de datos

In [13]:
data_bef, data_fee, data_aft = read_data('data/output_points_1_no_whiskers.xlsx')

In [14]:
# Split data into flower and bat data; before, during, and after feeding
flower_bef, flower_fee, flower_aft, bat_bef, bat_fee, bat_aft = split_data(data_bef, data_fee, data_aft)

# Show the data
print(bat_fee.head())

# Concatenate the data
flower = pd.concat([flower_bef, flower_fee, flower_aft])
bat = pd.concat([bat_bef, bat_fee, bat_aft])
data = pd.concat([flower, bat])

      x    y      z
9   281   94  349.0
10  283   98  345.0
11  283  100  341.0
12  283  103  338.0
13  288  103  336.0


### Curvatura

In [15]:
df = flower_fee.copy()

In [16]:
# Convertir a arrays de numpy
x = np.array(df['x'])
y = np.array(df['y'])
z = np.array(df['z'])

# Calcular las derivadas usando diferencias finitas
dx = np.gradient(x)
dy = np.gradient(y)
dz = np.gradient(z)

ddx = np.gradient(dx)
ddy = np.gradient(dy)
ddz = np.gradient(dz)

# Calcular la curvatura
curvatura = np.zeros(len(x))

for i in range(len(x)):
    # Vectores de la primera y segunda derivada
    r_prime = np.array([dx[i], dy[i], dz[i]])
    r_double_prime = np.array([ddx[i], ddy[i], ddz[i]])
    
    # Producto cruzado de r' y r''
    cross_product = np.cross(r_prime, r_double_prime)
    
    # Magnitud de r'
    norm_r_prime = np.linalg.norm(r_prime)
    
    # Curvatura
    curvatura[i] = np.linalg.norm(cross_product) / (norm_r_prime**3)

# Añadir la curvatura al DataFrame
df['curvatura'] = curvatura

# Desviación Estándar de la curvatura
std_curvatura = np.std(curvatura)
print('Desviación Estándar de la Curvatura:', std_curvatura)

# Media de la curvatura
mean_curvatura = np.mean(curvatura)
print('Media de la Curvatura:', mean_curvatura)


Desviación Estándar de la Curvatura: 1.8368621915524173
Media de la Curvatura: 0.5475373323457154


### Longitud

In [17]:
# Longitud de la trayectoria
longitud = np.zeros(len(x))

for i in range(1, len(x)):
    longitud[i] = longitud[i-1] + np.sqrt((x[i] - x[i-1])**2 + (y[i] - y[i-1])**2 + (z[i] - z[i-1])**2)

# Añadir la longitud al DataFrame
df['longitud'] = longitud

# Longitud total
longitud_total = longitud[-1]
print('Longitud Total:', longitud_total)

Longitud Total: 375.82886152263075


### Variación en cada eje

In [18]:
# Longitud de la trayectoria en el eje x (máximo y mínimo)
longitud_x = np.max(x) - np.min(x)
print('Longitud de la trayectoria en el eje x:', longitud_x)

# Longitud de la trayectoria en el eje y (máximo y mínimo)
longitud_y = np.max(y) - np.min(y)
print('Longitud de la trayectoria en el eje y:', longitud_y)

# Longitud de la trayectoria en el eje z (máximo y mínimo)
longitud_z = np.max(z) - np.min(z)
print('Longitud de la trayectoria en el eje z:', longitud_z)

# Volúmen del espacio
volumen = longitud_x * longitud_y * longitud_z
print('Volumen del Espacio:', volumen)

Longitud de la trayectoria en el eje x: 14
Longitud de la trayectoria en el eje y: 15
Longitud de la trayectoria en el eje z: 17.0
Volumen del Espacio: 3570.0
